# Draft `pyGIMLi(emg3d)`

**NEEDS**
- `pyGIMLi`
- `emg3d`
- `discretize`
- `xarray`
- `h5py`


An attempt at using `pyGIMLi` as an inversion framework for `emg3d` computations.

For developping purposes, we take a very simple grid/model/survey:
- Coarse mesh, no stretching (potentially too small).
- Simple double-halfspace model water-subsurface with a resistive block.
- Survey: A single 2D line, 6 sources, 1 frequency.

=> For this dev-implementation we also do inversion crime, using the same grid for forward modelling and inversion.

In [ ]:
import emg3d
import numpy as np
import pygimli as pg
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, SymLogNorm

In [ ]:
%matplotlib notebook

## Load survey (incl. data) and initial model

In [ ]:
data = emg3d.load('pginv.h5')
survey = data['survey']
model = data['model']
grid = model.grid

## Jacobian Wrapper

In [ ]:
class EMG3DJacobian(pg.Matrix):
    
    def __init__(self, sim):
        super().__init__()
        self.sim = sim

    def cols(self):
        # sim.model.size corresponds to the number of cells
        return self.sim.model.size

    def rows(self):
        # sim.survey.count corresponds to the number of non-NaN data points.
        return self.sim.survey.count

    def mult(self, x):
        """J * x """
        
        # If x is a vector of size model.size, we must reshape it to shape (nx, ny, nz)
        # (isotropic case; we can expand this later to VTI and triaxial isotropy)
        return self.sim.jvec(x.reshape(self.sim.model.shape, order='F'))

    def transMult(self, x):
        """J.T * x = (x * J.T)^T """
        
        # x must be a vector with the shape of the data, (nsrc, nrec, nfreq).
        # That might be fiddly, just ping me to help with that.
        return self.sim.jtvec(x)

## Forward Wrapper

In [ ]:
class MyFWD(pg.Modelling):
    
    def __init__(self, sim):
        self.super()
        
        # Store the simulation
        # (I replaced «fop» by «sim».)
        self.sim = sim
        
        # Do we need to translate the discretize mesh into a pg-mesh?
        # The cell widths hx;hy;hz and origin would be given by:
        # - hx: sim.model.grid.h[0]
        # - hy: sim.model.grid.h[1]
        # - hz: sim.model.grid.h[2]
        # - origin: sim.model.grid.origin
        # 
        # The grid in emg3d is always an, optionally stretched, TensorMesh.
        # As such, hx, hy, hz, and origin is enough to build it.
        self.mesh = pg.createGrid(...)
        
        self.J = EMG3DJacobian(self)
        self.setJacobian(self.J)

    def response(self, model):
        # do a lot of checks and cleanups
        
        # Clean emg3d-simulation, so things are recomputed
        self.sim.clean('computed')
        
        # Replace model
        self.sim.model = model
        
        # Compute responses for new model
        self.sim.compute()
        
        # Return the responses
        return self.sim.data.synthetic.data
    
    # I assume, for the misfit you also need to access the observed data:
    # => obs = self.sim.data.observed.data
    #
    # There can be observed data that have NaN's.
    # These src-rec-freq pairs are to be ignored.

    def createJacobian(self, model):
        pass  # do nothing

## Run an inversion

In [ ]:
# Create an emg3d Simulation instance
emg3d_sim = emg3d.simulations.Simulation(
    survey=survey,
    model=model,
    gridding='same',  # I will like to make that more flexible in the future
    max_workers=6,    # Adjust as needed
    receiver_interpolation='linear',  # Currently necessary for the gradient
    # solver_opts,
    tqdm_opts=False,  # Switch off verbose progress bars
)

fop = MyFWD(emg3d_sim)
INV = pg.Inversion(fop=fop)

# Regularization: Setting active/passive cells would be great,
# e.g., de-activating air and water for the inversion.
# And of course other regularizations (smoothness etc).
# INV.setRegularization(limits=[], correlationLengths=[...])

model = INV.run(dataVector, errorVector)

In [ ]:
emg3d.Report(['pygimli', 'pgcore'])